# Representing Orbits

Any point in any gravitational field can be described by an `AbstractOrbit` for the 
purposes of `GeneralAstrodynamics`. Every `AbstractOrbit` should provide the 
_state_ of the body in the gravitational field, and a _system_ to construct the 
gravitational field. The `AstrodynamicalModels` package provides representations
for orbits in common astrodynamical systems, e.g. the restricted two-body problem,
the circular restricted three-body problem, and the N-body problem. To illustrate 
how orbits are represented, let's walk through an example in the restricted 
two-body problem. In this scenario, we can describe a massless particle in the 
gravitational field of a spherical shell with mass $m$.

## Dependency Management

The following dependencies are used in this notebook.


In [22]:
import Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.develop(; url=joinpath(@__DIR__, "..", ".."))
Pkg.instantiate()

using GeneralAstrodynamics, DynamicQuantities, StaticArrays

  Activating project at `~/Projects/Astrodynamics/GeneralAstrodynamics.jl/paper`
   Resolving package versions...
  No Changes to `~/Projects/Astrodynamics/GeneralAstrodynamics.jl/paper/Project.toml`
  No Changes to `~/Projects/Astrodynamics/GeneralAstrodynamics.jl/paper/Manifest.toml`


## State Vectors

One of Julia's most popular packages --- `StaticArrays.jl` --- provides as 
`FieldVector` type that allows users to create performant labeled vectors. The
`AstrodynamicalModels.jl` package provides `FieldVector` sub-types for common
state and parameter representations in astrodynamics. Two such state 
representations are the `CartesianState` and `OrbitalElements`. Both types 
provide convenience constructors with keyword arguments, and allocations
without initialization.

The `CartesianState` type represents a position and velocity in three-dimensional
space: $x$, $y$, $z$, $\dot{x}$, $\dot{y}$, and $\dot{z}$. Note the convenience 
constructors shown below. This type, and all other state vector types provided by
`AstrodynamicalModels`, is `mutable`. 

In [23]:
CartesianState(x=11e5, z=5e5, ẏ=1e4)

CartesianState with eltype Float64

  x: 1.1e6
  y: 0.0
  z: 500000.0
  ẋ: 0.0
  ẏ: 10000.0
  ż: 0.0


In [24]:
CartesianState()

CartesianState with eltype Float64

  x: 0.0
  y: 0.0
  z: 0.0
  ẋ: 0.0
  ẏ: 0.0
  ż: 0.0


In [25]:
CartesianState(undef)

CartesianState with eltype Float64

  x: 3.0665334336e-314
  y: 5.0e-324
  z: 1.5714e-317
  ẋ: 5.0e-324
  ẏ: 4.0e-323
  ż: NaN


The `OrbitalElements` type defines a Keplerian state with traditional orbital elements: 
eccentricity $e$, semi-major axis $a$, inclination $i$, right ascension of the ascending
node $\Omega$, argument of periapsis $\omega$, and true anomaly $\nu$. 

In [26]:
OrbitalElements(e=0.99, a=10e5)

OrbitalElements with eltype Float64

  e: 0.99
  a: 1.0e6
  i: 0.0
  Ω: 0.0
  ω: 0.0
  ν: 0.0


In [27]:
OrbitalElements()

OrbitalElements with eltype Float64

  e: 0.0
  a: 0.0
  i: 0.0
  Ω: 0.0
  ω: 0.0
  ν: 0.0


In [28]:
OrbitalElements(undef)

OrbitalElements with eltype Float64

  e: 2.03e-322
  a: 4.96255345484001e228
  i: 6.807402956851786e199
  Ω: 9.701582850788981e189
  ω: 2.1162700550059405e214
  ν: 6.2145277554479345e175


## Parameter Types

Dynamical systems may be defined by their equations, states, and parameters. 
The parameters describe how the specific instantiation of a general system 
will behave. Parameter types for all systems provided by `AstrodynamicalModels.jl`
are available as `FieldVector` sub-types, though this may change (with a major
version bump) in the future to better conform with `ModelingToolkit.jl`'s 
`v9` release. All parameter types are **not mutable**, therefore they have no 
uninitialized memory constructors. 

:::{.callout-note}

This interface may look familiar to you! We are following the SciML organization's
_solver interface_, where any `ODESystem` is defined (mostly) by its states,
parameters, and equations. 

:::


In [29]:
R2BParameters(5e11)

R2BParameters with eltype Float64

  μ: 5.0e11


In [30]:
R2BParameters(μ=1e7)

R2BParameters with eltype Float64

  μ: 1.0e7


## Dynamics

Within `AstrodynamicalModels.jl`, dynamics are mapped one-to-one with parameter
types. No two systems may have the same parameter types! To recover an `ODEFunction`
with a system's equations of motion, try the `dynamics` method.

In [31]:
rand(CartesianState)

CartesianState with eltype Float64

  x: 0.9560323290768354
  y: 0.6378002185832633
  z: 0.733863458978701
  ẋ: 0.3759268268204885
  ẏ: 0.8662613119506547
  ż: 0.4776611433305632


In [32]:
earth = R2BParameters(5e11)

f = AstrodynamicalModels.dynamics(earth)

du = let u = randn(CartesianState), p = randn(R2BParameters), t = NaN
    du = zeros(6)
    f(du, u, p, t)
    du
end

6-element Vector{Float64}:
 -0.20047416524420056
 -0.23012851587920408
 -0.8927694627193575
  0.060723333253600464
  0.013593251974973165
  0.1830599389748437

## Orbit Types

Finally, we have all of the pieces necessary to fully describe an astrodynamical
orbit. See the `Orbit` type's documentation below, along with usage examples.
Keeping the state and parameters tied together makes domain-specific calculations
easier: for this, see the `AstrodynamicalCalculations` package! 

When a `R2BOrbit` is shown within Julia, its conic section is automatically
computed and displayed. 

In [33]:
orbit = Orbit(
    CartesianState(x=11e5, z=3e5, ẏ=1e3),
    R2BParameters(μ=5e17)
)

Elliptical Orbit in Restricted Two Body Dynamics

  CartesianState with eltype Float64
  
    x: 1.1e6
    y: 0.0
    z: 300000.0
    ẋ: 0.0
    ẏ: 1000.0
    ż: 0.0

  R2BParameters with eltype Float64
  
    μ: 5.0e17


In [34]:
f = dynamics(orbit)
f(du, rand(CartesianState), rand(R2BParameters), 0)
du

6-element Vector{Float64}:
  0.5688870324806231
  0.20773839389773607
  0.6107848389898091
 -0.7392318217719731
 -0.21888630690684702
 -0.49614552704070186

In [35]:
eccentricity(orbit)

0.99999771964915

In [36]:
cartesian_to_keplerian(orbit.state, orbit.parameters)

OrbitalElements with eltype Float64

  e: 0.99999771964915
  a: 570088.3625503101
  i: 0.2662520491509256
  Ω: 4.71238898038469
  ω: 4.71238898038469
  ν: 3.141592653589793
